In [37]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

import sys
sys.path.insert(0, '../../')

import importlib
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
import os
import pandas as pd
import json

from coralme.builder.main import MEBuilder

from Bio import Entrez, SeqIO
Entrez.email = 'cdalldorf@sdsu.edu'

# Function to download GenBank file
def download_genbank(ref_id, output_dir, verbose = False):
    try:
        if verbose: print(f"Fetching {ref_id}...")
        with Entrez.efetch(db="nucleotide", id=ref_id, rettype="gb", retmode="text") as handle:
            gb_record = handle.read()
            output_file = os.path.join(output_dir, 'genome.gb')
            with open(output_file, "w") as f:
                f.write(gb_record)
        if verbose: print(f"Saved {ref_id} to {output_file}")
        return(1)
    except Exception as e:
        if verbos: print(f"Error fetching {ref_id}: {e}")
        return(0)

ModuleNotFoundError: No module named 'coralme.solver'

In [42]:
import pandas as pd
import os
import cobra
import json

from Bio import Entrez, SeqIO
Entrez.email = 'cdalldorf@sdsu.edu'

# Function to download GenBank file
def download_genbank(ref_id, output_dir, verbose = False):
    try:
        if verbose: print(f"Fetching {ref_id}...")
        with Entrez.efetch(db="nucleotide", id=ref_id, rettype="gb", retmode="text") as handle:
            gb_record = handle.read()
            output_file = os.path.join(output_dir, 'genome.gb')
            with open(output_file, "w") as f:
                f.write(gb_record)
        if verbose: print(f"Saved {ref_id} to {output_file}")
        return(1)
    except Exception as e:
        if verbos: print(f"Error fetching {ref_id}: {e}")
        return(0)

In [59]:
# go through each line, convert model, find files necessary, generate input jsons
base_dir = '../species_files/Pseudomonas_files/'
info_df = pd.read_excel(base_dir+'Strain_models_genome-information_predictedM9.xlsx', index_col = 0)
for index, row in info_df.iterrows():
    if index == 'Reference': continue

    # make directory
    out_dir = base_dir+'/individual_species/'+index+'/'
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    if not os.path.exists(out_dir+'/inputs'):
        os.mkdir(out_dir+'/inputs')
    if not os.path.exists(out_dir+'/outputs'):
        os.mkdir(out_dir+'/outputs')
    
    # pull genome.gb file from NCBI
    if not os.path.exists(out_dir+'/inputs/genome.gb'):
        download_genbank(index, out_dir+'/inputs')

    # TODO - automatic download of Biolog files
    # you tried this before without success, perhaps do manually

    # let's create organism.json and input.json files
    input_json = {
      "m-model-path": "./inputs/model.json",
      "genbank-path": "./inputs/genome.gb",
      "biocyc.genes": "./inputs/genes.txt",
      "biocyc.prots": "./inputs/proteins.txt",
      "biocyc.TUs": "./inputs/TUs.txt",
      "biocyc.RNAs": "./inputs/RNAs.txt",
      "biocyc.seqs": "./inputs/sequences.fasta",
      "df_gene_cplxs_mods_rxns": "./outputs/building_data/automated-org-with-refs.xlsx",
      "out_directory": "./outputs",
      "log_directory": "./outputs",
      "run_bbh_blast": True,
      "e_value_cutoff": 1e-10,
      "dev_reference": True,
      "include_pseudo_genes": True,
      "locus_tag" : "locus_tag"
    }
    organism = {
    	"ME-Model-ID" : row['organism'],
    	"growth_key" : "mu1",
    	"complex_cofactors" : {},
    	"lipoprotein_precursors" : {},
    	"lipid_modifications" : [],
    	"flux_of_biomass_constituents" : {},
    	"flux_of_lipid_constituents" : {},
    	"braun's_lipoprotein" : [],
    	"braun's_lpp_flux" : -0.0,
    	"braun's_murein_flux" : -0.0,
    	"gr_data_doublings_per_hour" : [0, 0.6, 1.0, 1.5, 2.0, 2.5],
        "percent_dna_data" : [0.0592, 0.0512, 0.0330, 0.0252, 0.0222, 0.0208],
    	"defer_to_rxn_matrix" : []
    }
    with open(out_dir+'organism.json', 'w') as f:
        json.dump(organism, f, indent = 4)
    with open(out_dir+'input.json', 'w') as f:
        json.dump(input_json, f, indent = 4)

    
    # convert model file to json if necessary
    matlab_file = base_dir+'matlab-strain-models/'+index+'.mat'
    json_file = out_dir+'/inputs/model.json'
    if not os.path.exists(json_file) and os.path.exists(matlab_file):
        model = cobra.io.load_matlab_model(matlab_file)
        cobra.io.save_json_model(model, json_model)
        fa = fo

    # create the model
    organism = out_dir+'organism.json'
    inputs = out_dir+'input.json'
    builder = MEBuilder(*[organism, inputs])
    builder.generate_files(overwrite=True)
    builder.build_me_model(overwrite=True)
    
    # gapfill the model
    builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })
        

GurobiError: License expired 2023-10-25